# Create the DataPacks
Use the built-in Data Workbench functionality in Bloomberg Lab to retrieve the datasets needed to backtest the different strategies and construct the portfolios in Signal Lab. The datasets requested are only those needed for this purpose. Financial Statement datasets are retrieved separately to this. Datasets added into the datapack are:

- Index membership and weights
- Trading Calendars
- Current market cap
- Price
- Total Returns
- Beta
- BICS sector
- Analyst recommendations for the base study

In [1]:
# Install signal_lab if it isn't already loaded
%package install bloomberg.bquant.signal_lab=1.5.2

Running: micromamba install bloomberg.bquant.signal_lab=1.5.2 --yes --log-level=error

  Package                                                   Version  Build                   Channel          Size
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

  + icu                                                        75.1  he02047a_0              conda-forge      12MB
  + hmmlearn                                                  0.3.3  py39hf88036b_0          conda-forge     177kB
  + lz4                                                       4.3.3  py39hd7df8f7_1          conda-forge      37kB
  + fontconfig                                               2.15.0  h7e30c49_1              conda-forge     266kB
  + pixman                                                   0.44.2  h29eaf8c_0             

In [1]:
import os
import pandas as pd
import s3fs
import bql

from bloomberg.bquant.signal_lab.data_workbench import (
    create_data_pack,
    load_data_pack,
    DataPack,
    DataPackConfig,
    StorageType,
    BQLIndexUniverse,
    BQLIndexWeight,
    BQLData,
    ListUniverse,
    TradingCalendarProxy,
    FetchMode,
    FetchErrorHandling
)

from bloomberg.bquant.signal_lab.workflow.utils import get_sandbox_path

In [2]:
# Create the BQL object to request the data
bq = bql.Service()

In [3]:
datapack_path = f"{get_sandbox_path()}/tmp/fs/datapack"

## Define the universe

In [4]:
index_name = 'INDU Index'
data_pack_config = DataPackConfig(pd.Timestamp("2019-03-31"),pd.Timestamp("2025-03-31"))

universe_definitions = {
    index_name: BQLIndexUniverse(index_name)
}

### Define the Data items

In [5]:
data_item_definitions = {
    "index_weight": BQLIndexWeight(),
    "trading_calendar": TradingCalendarProxy(),

    "analyst_rating": BQLData(bq.data.best_analyst_rating()),
    "target_price": BQLData(bq.data.best_target_price()),

    "cur_mkt_cap": BQLData(bq.data.cur_mkt_cap()),
    "day_to_day_tot_return_gross_dvds": BQLData(bq.data.day_to_day_tot_return_gross_dvds()),
    "beta": BQLData(bq.data.beta(), rolling_date=True, freq="w"),
    "px_last": BQLData(bq.data.px_last()),

    'bics_level_1': BQLData(bq.data.classification_name("bics", "1"), rolling_date=True),
    'bics_level_2': BQLData(bq.data.classification_name("bics", "2"), rolling_date=True),
    'bics_level_3': BQLData(bq.data.classification_name("bics", "3"), rolling_date=True),
    'bics_level_4': BQLData(bq.data.classification_name("bics", "4"), rolling_date=True),
}

### Create the DataPack

In [6]:
data_pack = create_data_pack(
    data_pack_config = data_pack_config,
    universe_definitions=universe_definitions,
    data_item_definitions=data_item_definitions,
    storage_type=StorageType.PARQUET,
    storage_path=datapack_path,
    overwrite_existing_data_pack=True
)

In [7]:
data_pack.get_fetch_status()

,ID,TYPE,IS_LOADED,LOAD_TIME
0,INDU Index,UNIVERSE,False,NaT
1,index_weight,DATA_ITEM,False,NaT
2,trading_calendar,DATA_ITEM,False,NaT
3,analyst_rating,DATA_ITEM,False,NaT
4,target_price,DATA_ITEM,False,NaT
5,cur_mkt_cap,DATA_ITEM,False,NaT
6,day_to_day_tot_return_gross_dvds,DATA_ITEM,False,NaT
7,beta,DATA_ITEM,False,NaT
8,px_last,DATA_ITEM,False,NaT
9,bics_level_1,DATA_ITEM,False,NaT


### Fetch the data

In [8]:
fetch_status = data_pack.run_fetch()

2025-04-25 09:00:45,969 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching universes
2025-04-25 09:00:46,690 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - universe fetching summary: INDU Index
2025-04-25 09:00:46,692 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching universe: INDU Index
2025-04-25 09:00:46,693 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.definitions.bql_index_universe - fetching BQL index universe: INDU Index


  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

2025-04-25 09:01:09,465 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching data items
2025-04-25 09:01:10,386 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - data items fetching summary: index_weight, trading_calendar, analyst_rating, target_price, cur_mkt_cap, day_to_day_tot_return_gross_dvds, beta, px_last, bics_level_1, bics_level_2, bics_level_3, bics_level_4
2025-04-25 09:01:10,791 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching data item: index_weight
2025-04-25 09:01:10,792 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.definitions.bql_index_weight - fetching BQL index weight: INDU Index


  0%|          | 0/37 [00:00<?, ?it/s]

...

2025-04-25 09:04:24,703 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching data item: bics_level_3


  0%|          | 0/37 [00:00<?, ?it/s]

2025-04-25 09:04:42,764 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching data item: bics_level_4


  0%|          | 0/37 [00:00<?, ?it/s]

2025-04-25 09:05:01,334 - (140594856351552) - INFO - external.bloomberg.bquant.signal_lab.data_workbench.data_pack - fetching completed


In [9]:
data_pack.get_fetch_status()

,ID,TYPE,IS_LOADED,LOAD_TIME
0,INDU Index,UNIVERSE,True,2025-04-25 09:01:09.160193
1,index_weight,DATA_ITEM,True,2025-04-25 09:01:38.926912
2,trading_calendar,DATA_ITEM,True,2025-04-25 09:01:56.920008
3,analyst_rating,DATA_ITEM,True,2025-04-25 09:02:15.311967
4,target_price,DATA_ITEM,True,2025-04-25 09:02:34.602782
5,cur_mkt_cap,DATA_ITEM,True,2025-04-25 09:02:55.288841
6,day_to_day_tot_return_gross_dvds,DATA_ITEM,True,2025-04-25 09:03:14.579476
7,beta,DATA_ITEM,True,2025-04-25 09:03:30.089529
8,px_last,DATA_ITEM,True,2025-04-25 09:03:48.877069
9,bics_level_1,DATA_ITEM,True,2025-04-25 09:04:06.786431


In [16]:
datapack_path = f"{get_sandbox_path()}/tmp/fsa/datapack"

In [17]:
new_data_pack = load_data_pack(storage_path=datapack_path, storage_type=StorageType.PARQUET)

ValueError: There is no DataPack found in the specific path 's3://awmgd-prod-finml-02-sandbox-user/bclarke16/tmp/fsa/datapack'

In [15]:
new_data_pack.get_fetch_status()

,ID,TYPE,IS_LOADED,LOAD_TIME
0,INDU Index,UNIVERSE,True,2025-04-25 09:01:09.160193
1,index_weight,DATA_ITEM,True,2025-04-25 09:01:38.926912
2,trading_calendar,DATA_ITEM,True,2025-04-25 09:01:56.920008
3,analyst_rating,DATA_ITEM,True,2025-04-25 09:02:15.311967
4,target_price,DATA_ITEM,True,2025-04-25 09:02:34.602782
5,cur_mkt_cap,DATA_ITEM,True,2025-04-25 09:02:55.288841
6,day_to_day_tot_return_gross_dvds,DATA_ITEM,True,2025-04-25 09:03:14.579476
7,beta,DATA_ITEM,True,2025-04-25 09:03:30.089529
8,px_last,DATA_ITEM,True,2025-04-25 09:03:48.877069
9,bics_level_1,DATA_ITEM,True,2025-04-25 09:04:06.786431
